In [2]:
import numpy as np
import pandas as pd
import os
import sys
import tensorflow as tf
import xgboost as xgb
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection  import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_log_error
from sklearn.externals import joblib
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost.sklearn import XGBRegressor
import LightGBM as lgb
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import Lasso

In [234]:
### 读取train_set的数据，包含 p1,p2
train_set = pd.read_csv('./data/train_set.csv',encoding='gbk')

/Users/lmlyz/Library/Python/3.6/lib/python/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (89,92,93,94,95,96,100,101,102,103,105,122,124,125,129,130,131,132,133,134,135,136,137,144,145,149,150,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,178,182,183,193,198,199,210,214,216,217,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [235]:
###读取 part2的数据
df_part2 = pd.read_csv('./data/part2.csv',sep='$')

In [236]:
## 读取test_set的数据 特征和train_set 对应 包含 p1,p2的数据
test_set = pd.read_csv('./data/test_set.csv',encoding='gbk')

/Users/lmlyz/Library/Python/3.6/lib/python/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (92,93,94,95,96,101,103,104,105,122,124,125,129,130,131,133,134,135,136,137,144,145,150,152,162,182,193,214,216,217,218,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [244]:
part2_col = list(set(df_part2['table_id']))

In [39]:
print(train_set.shape,test_set.shape)

(38199, 245) (9538, 245)


In [238]:
###设置每张表的 index 为 vid
def resetIndex(df):
    vid = df['vid']
    df = df.drop('vid',axis=1)
    df = df.set_index(vid)
    return df

In [239]:
col = ['收缩压','舒张压','血清甘油三酯','血清高密度脂蛋白','血清低密度脂蛋白']

In [241]:
train_set = resetIndex(train_set)
test_set = resetIndex(test_set)

In [245]:
### 过滤掉part1的数据 保留需要预测的数据
def filter_col(df,col,part2_col):
    
    for feature in df.columns:
        if feature not in part2_col:
            if feature not in col:
                df = df.drop(feature,axis=1)
        
    return df
    

In [246]:
#过滤掉p1的数据，只保留p2的数据
train_set = filter_col(train_set,col,part2_col)

In [247]:
test_set = filter_col(test_set,col,part2_col)

In [287]:
### 将train_set 和 test_set 合并成一张表，包含了 [收缩压，舒张压,....]等5列
df_all = train_set
df_all = df_all.append(test_set)

In [256]:
df_all.to_csv('./temp/df_all.csv',encoding='gbk')

In [266]:
## 删掉test_set中没有在train_set中出现的特征
def drop_col(test_set,train_set):
    for col in test_set.columns:
        if col not in train_set.columns:
            test_set = test_set.drop(col,axis=1)
    print(test_set.shape)
    return test_set

In [289]:
## 某个特征的缺失值大于75的话就删除这个特征
def drop_col_thresh(df,thresh = 0.25):
    
    num_rows = len(df.index)
    cols = []
    for item in df.columns:
        if float(df[item].count()) / num_rows < thresh:
            cols.append(item)
            
    print(len(cols))
    
    df = df.drop(cols,axis=1)
    #pass
    return df

In [443]:
import re
##处理阴性 阳性 ++ --这种类型的列数据  ['100010','3190','3191','3192','3195','3196','3197','3430']
def zz_col_1(item):
    p = r"[0-9A-Za-z/()\s\.>=]*"
    

    item = str(item)
    tmp=re.sub(p,"",item)
    if tmp == "":
        return item
    tmp = tmp.replace("阴性","-").replace("阳性","+").replace("未做","").replace("未见","").replace("１＋","+")\
    .replace("２＋","+").replace("１＋","+").replace("血液","").replace("+-","+").replace("++","+")\
    .replace("+++","+").replace("--","-").replace("透明","").replace("正常","-")
  
    
    return tmp
##处理带单位   :    /hp   /Hp  /Lp  个  阳性(+)  偶见  未检出     ['3429','300005','3430']


def zz_col_2(item):
    p=r"[^0-9\-.]"
    item = str(item)
    temp = re.sub(p,"",item)
    
    if temp == "":
        return item
    
 
    if temp.find("-")!=0:
        li = temp.split("-")
        try:
            temp = str((float(li[0])+float(li[1]))/2)
        except:
#                 print(li)
            if len(li)>2:
                temp = str((float(li[0])+float(li[2]))/2)
            else:
                temp=li[0]
    return temp
##   ['2228','2229']   ## （阴性 0.988）
def zz_col_3(item):
    p = r"[a-z\s()（]" #过滤 a-z\s()（
    p1 = r"[0-9\.]" #过滤 数字型，用于保留原来的数值型数据
    
    tmp = re.sub(p,"",str(item))
    tmp1 = re.sub(p1,"",tmp)
    if(tmp1 == ""):
        return tmp
    tmp = tmp.replace("++","+").replace("+-","+").replace("阳","+").replace("阴","-").replace("性","").replace("极","").replace("重","")\
                .replace("度","").replace("弱","")
       
    return tmp



In [389]:
def func4(item):   ##过滤掉 0.9770.977 返回0.977
    item = str(item)
    if len(item.split('.'))>2:
        i =item.rindex('.')
        item = item[0:i-1]
    return item
    

In [444]:
df_all_tmp1 = df_all

In [445]:
df_all_tmp1 = df_all_tmp1.applymap(zz_col_1)
df_all_tmp1 = df_all_tmp1.applymap(zz_col_2)
df_all_tmp1 = df_all_tmp1.applymap(zz_col_3)
df_all_tmp1 = df_all_tmp1.applymap(func4)

In [448]:
##这里所有的数据只包含 p2中的数值型 和 + -类型，没有做缺失值的填充
df_all_tmp1.to_csv('./temp/df_all_tmp3.csv',encoding='gbk')

#以上完成了p2的数据清洗。。。

In [ ]:
#########################################################

In [294]:
def clean_label(x):
    x = str(x)
    
  
    if len(x.split(sep='.'))>2:#2.2.8
        i = x.rindex('.')
        x = x[0:i]+x[i+1:]
    
    if '。' in x:
        x = x.replace('。','.')
    if ', ' in x:
        x = x.replace(', ','.')
    if x == ' ' or x == '()':
        x = np.nan
    if '标本已退' in str(x) or '未做' in str(x) or '未查' in str(x) or '弃查' in str(x):
        x = np.nan
    if str(x).isdigit()==False and len(str(x))>4:
        x = x[0:4]
    if 'A' in str(x):
        x = np.nan
        
    if 'CLT' in str(x):
        x = np.nan
        
      
    return x
    

In [296]:
import re
def func1(x):
    #regexp = re.compile(r'[^\x00-\x7f]') ##查找中文
    x = str(x)
    #if(regexp.search(x)):
        #x = np.nan  ##如果有中文设置成 np.nan
    if '----' in str(x):
        x = np.nan
    if '....' == str(x):
        x = np.nan
    if str(x) == '':
        x = np.nan
    if str(x) == '.':
        x = np.nan
    if '.()' in str(x):
        x = np.nan
    return x

In [299]:
def func2(x):
    x = str(x)
    if '>' in x:#> 11.00
        i = x.index('>')
        x = x[i+1:]
    if '<' in x:
        i = x.index('<')
        x = x[i+1:]
    if '=' in x:
        i = x.index('=') # =1.
        x = x[i+1:]
    if ' ' in x:
        x = x.replace(' ','.') # 50 5
    if '-' in x or '+' in x: # + or - 出现在特征中直接置为 np.nan
        x = np.nan
    if '(' in str(x):
        i = x.index('(') # 125(
        x = x[:i]
    return x

In [309]:
# df_all_tmp1 = df_all_tmp1.applymap(lambda x:func1(x))
# df_all_tmp1 = df_all_tmp1.applymap(lambda x:func2(x))

In [401]:
#df_all_tmp1.head()
#df_all_tmp1 = pd.read_csv('./temp/df_all_tmp1.csv',encoding='gbk')

In [269]:
#test_set1_tmp1 = test_set.applymap(clean_label)

In [313]:
#train_set = pd.read_csv('./temp/train_set_p2_cleaned.csv',encoding='gbk')

/Users/lmlyz/Library/Python/3.6/lib/python/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (13,26,27,32,36,48,63,65,67,72,73,84) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [31]:
def drop_row(df,thresh = 0.35):
    num_rows = len(df.index)
    num_cols = len(df.columns)
    rows = []
    for i in range(num_rows):
        if float(df.iloc[i].count()) / num_cols < thresh:
            rows.append(df.index[i])
            
    print(len(rows))
    df = df.drop(rows,axis=0)
    
    return df

In [184]:
def drop_with_mean(df):
    rows = []
    col_mean = {}
    for item in df.columns:
        col_mean[item] = df[item].mean()
    
    for i in range(len(df.index)):
        for k,v in df.iloc[i].items():
            if np.isnan(v) == False:
                if v > 3.0*col_mean[k]:
                    rows.append(df.index[i])
    print(len(rows))
    
    df = df.drop(rows,axis=0)
    
    return df

In [575]:
def norm(df,col,flag = True):
    scalar = StandardScaler()
    if flag:
        y = df[col]
        t = df.drop(col,axis=1)
        scalar.fit(t)
        df2 = scalar.transform(t)
        X = pd.DataFrame(data=df2,index=df.index,columns = t.columns)
        X[col]= y
    else:
        t = df
        scalar.fit(t)
        df2 = scalar.transform(t)
        X = pd.DataFrame(data=df2,index=df.index,columns = t.columns)
    return X